# Batch Learning

# Import Libraries

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

import numpy as np
from rich import print

import pandas as pd

## Split data

In [2]:
data = pd.read_csv("./air.csv")
targets = data.pop("T")

data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,88.700000,2.231000


In [3]:
# TODO: Test if its really necessary to do the preprocess

encoder = LabelEncoder()

data["Time"] = encoder.fit_transform(data["Time"])
# data["Month"] = data.Date.map(lambda x: int(x.split("/")[1]))
# data["Season"] = (data.Month % 12)//3
# data["Time_cos"] = data.Time.map(np.cos)
# data["Time_sin"] = data.Time.map(np.sin)

# data = pd.get_dummies(data, columns = ["Month", "Season"])
# data = pd.get_dummies(data, columns = ["Season"])
data.drop(columns = ['Date'], inplace = True)


In [4]:
data.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH
0,18,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578
1,19,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255
2,20,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502
3,21,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867
4,22,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888


In [5]:
len_train_set = int(len(targets) * 0.75)

X_train = data[:len_train_set]
X_test = data[len_train_set:]
y_train = targets[:len_train_set]
y_test = targets[len_train_set:]

# Pipeline Ana

In [22]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from typing import Callable 
from sklearn.metrics import mean_absolute_error

class SequentialImpute(BaseEstimator, TransformerMixin):
    def __init__(self, detect: Callable = lambda x: x == -200):
        self.detect = detect 
        self.last = None 
        self.last_target = None
        
    def fit(self, X, y):
        X, y = map(np.array, (X, y))
        self.last = np.zeros(X.shape[1])
        self.last_target = 0
        self.X_ = X 
        self.y_ = y 
        for i in range(len(y)):
            x = np.where(self.detect(X[i]), self.last, X[i])
            self.last = x 
            if not self.detect(y[i]):
                self.last_target = y[i]
        return self 
    
    def transform(self, X):
        X = np.array(X)
        for i, x in enumerate(X):
            X[i] = np.where(self.detect(x), self.last, x)
            self.last = x
        return X

pip = Pipeline([
    ('imputer', SequentialImpute()), 
    ('scaler', StandardScaler()),
    ('model', KNeighborsRegressor())
])

pip.fit(X_train,y_train)
y_pred = pip.predict(X_test)
mean_absolute_error(y_test, y_pred)

15.84130769230769

In [25]:
y_test

7017     9.5
7018     9.8
7019    10.7
7020    10.4
7021     9.7
        ... 
9352    21.9
9353    24.3
9354    26.9
9355    28.3
9356    28.5
Name: T, Length: 2340, dtype: float64

In [30]:
pipeline = Pipeline([
    SequentialImpute(),
    StandardScaler(),
    RandomForestRegressor()
]
)
param_grid = {
    "randomforestregressor__n_estimators": [50, 100],
    "randomforestregressor__max_depth": [None, 10],
    "randomforestregressor__min_samples_split": [2, 5],
    "randomforestregressor__min_samples_leaf": [1, 2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")
print("Best model:", best_pipeline)
print("Best hyperparameters:", best_params)

ValueError: Invalid parameter 'randomforestregressor' for estimator Pipeline(steps=[SequentialImpute(), StandardScaler(), RandomForestRegressor()]). Valid parameters are: ['memory', 'steps', 'verbose'].